# 言語処理100本ノック
# 第8章: 機械学習

---

本章では，Bo Pang氏とLillian Lee氏が公開している`Movie Review Dataのsentence polarity dataset v1.0`を用い，文を肯定的（ポジティブ）もしくは否定的（ネガティブ）に分類するタスク（極性分析）に取り組む．

---

In [52]:
from tqdm import tqdm_notebook as tqdm
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [7]:
if os.path.exists("rt-polaritydata/"):
    print("rt-polaritydata/ has already been exist!")
else:
    os.system("wget -O rt-polaritydata.tar.gz http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz")
    os.system("tar -zxvf rt-polaritydata.tar.gz")
    
    

rt-polaritydata/ has already been exist!


In [10]:
ls rt-polaritydata/ -l

ls: -l: No such file or directory
rt-polaritydata/:
rt-polarity.neg  rt-polarity.pos


In [30]:
%%bash
head -3 rt-polaritydata/rt-polarity.pos

head -3 rt-polaritydata/rt-polarity.neg


the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 
the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth . 
effective but too-tepid biopic
simplistic , silly and tedious . 
it's so laddish and juvenile , only teenage boys could possibly find it funny . 
exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . 


In [31]:
%%bash

nkf -w --overwrite rt-polaritydata/*

### Helpers

In [93]:
def check(l: list, head=5) -> None:
    """
    Data Helper check()
    
    Params
    ------
    l: list
        list that you want to check

    head: int
        size, length of sample log output
    
    Return
    ------
    output sentence for checking list object
    structure and size.
    """
    output = ""
    output += "Length: {}\n".format(len(l))
    output += "First {} elements: {}".format(head, l[:head])
    print(output)
    return

def tokenize(sentence: str) -> list:
    return sentence.split(" ")

---

## 70. データの入手・整形

文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

`rt-polarity.pos`の各行の先頭に**"+1 "**という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
`rt-polarity.neg`の各行の先頭に**"-1 "**という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
`sentiment.txt`を作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [94]:
import random

def random_concat(pos="./rt-polaritydata/rt-polarity.pos",
                  neg="./rt-polaritydata/rt-polarity.neg",
                  seed=1, output_file_name="sentiment.txt") -> tuple:
    """
    70.「rt-polarity.posの各行の先頭に"+1 "という文字列を追加する
    （極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
    rt-polarity.negの各行の先頭に"-1 "という文字列を追加する
    （極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
    上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
    sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．」

    Params
    ------
    pos : str
        file path to pos-data text file
    neg : str
        negative one
    seed : int (default: 1)
        seed value for random function
    
    Return
    ------
    output : str
        randomly concatenated text.
    """
    
    labels = list()

    with open(pos, encoding="utf-8") as p:
        pos = ["+1" + " " + str(line.rstrip()) for line in p.readlines()]

    with open(neg, encoding="utf-8") as n:
        neg = ["-1" + " " + str(line.rstrip()) for line in n.readlines()]

    output = pos + neg
    
    random.seed(seed)
    random.shuffle(output)

    labels = [1] * len(pos) + [0] * len(neg)
    random.seed(seed)
    random.shuffle(labels)
    
    print("Randomized text list.")
    check(output, head=1)

    with open(output_file_name, "w") as w:
        w.writelines(output)

    return "\n".join(output), labels

In [95]:
sentiment, labels = random_concat()

# 正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．
print("positive file P={}".format(labels.count(1)))
print("negative file N={}".format(labels.count(0)))

Randomized text list.
Length: 10661
First 1 elements: ['-1 a negligible british comedy .']
positive file P=5330
negative file N=5331


In [96]:
# Test
# len(list(map(bool, labels)))
# pd.Series(sentiment.split("\n"))[(list(map(bool, labels)))]


---

## 71. ストップワード

英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

In [98]:
stop_words = ["at", "in", "on", "by", "with", "of",
              "I", "you", "am", "are", "it", "is",
              "this", "that", "these", "those", "the"]


def is_stop_word(word : str) -> bool:
    """
    英語のストップワードのリスト（ストップリスト）を適当に作成せよ．
    さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，
    それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．
    """
    return word in stop_words


def have_stop_word(word : list) -> bool:
    """
    英語のストップワードのリスト（ストップリスト）を適当に作成せよ．
    さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，
    それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．
    """
    if type(word) == str:
        word = tokenize(word)
    
    return (True in [stop_word in word for stop_word in stop_words])


def remove_stop_word(words : list) -> list:
    
    if type(words) == str:
        words = tokenize(words)
    
    for sw in stop_words:
        if sw in words:
            words.remove(sw)

    return words


# check it
print(is_stop_word("this"))
print(have_stop_word("the gorgeously elaborate continuation of"))
print(have_stop_word("gorgeously elaborate continuation"))
print(have_stop_word(remove_stop_word("the gorgeously elaborate continuation of")))

True
True
False
False


---

## 72. 素性抽出

極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

---

## 73. 学習

72で抽出した素性を用いて，ロジスティック回帰モデルを学習せよ．

### 74. 予測
73で学習したロジスティック回帰モデルを用い，与えられた文の極性ラベル（正例なら"+1"，負例なら"-1"）と，その予測確率を計算するプログラムを実装せよ．

### 75. 素性の重み
73で学習したロジスティック回帰モデルの中で，重みの高い素性トップ10と，重みの低い素性トップ10を確認せよ．

### 76. ラベル付け
学習データに対してロジスティック回帰モデルを適用し，正解のラベル，予測されたラベル，予測確率をタブ区切り形式で出力せよ．

### 77. 正解率の計測
76の出力を受け取り，予測の正解率，正例に関する適合率，再現率，F1スコアを求めるプログラムを作成せよ．

### 78. 5分割交差検定
76-77の実験では，学習に用いた事例を評価にも用いたため，正当な評価とは言えない．すなわち，分類器が訓練事例を丸暗記する際の性能を評価しており，モデルの汎化性能を測定していない．そこで，5分割交差検定により，極性分類の正解率，適合率，再現率，F1スコアを求めよ．


### 79. 適合率-再現率グラフの描画

ロジスティック回帰モデルの分類の閾値を変化させることで，適合率-再現率グラフを描画せよ．